In [ ]:
from depmapomics import mutations
import dalmatian as dm

In [ ]:
wgs_wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WGS_CN")
wgsmut = mutations.aggregateMAFs(wgs_wm)

In [ ]:
wes_wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WES_CN_hg38")
wesmut = mutations.aggregateMAFs(wes_wm)

In [ ]:
import pandas as pd

merged_maf = pd.concat([wgsmut, wesmut])

In [ ]:
merged_maf.to_csv("new_maf.csv", index=None)

In [ ]:
merged_maf

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

tc.update_dataset(
    changes_description="add new maf",
    dataset_permaname="comparison-new-maf-old-maf-24b4",
    upload_files=[            
        {
            "path": "new_maf.csv",
            "format": "TableCSV",
            "encoding": "utf-8",
        },
    ],
    upload_async=False,
    add_all_existing_files=True,
)

In [ ]:
old_wes_maf = pd.read_csv("gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/fe49d507-c56e-4b83-b8f5-a9ff65dc4521/aggregateMAFs_selectFields_workflow/e336ee4e-a444-4e82-a2a5-16906e5a91ea/call-aggregateMAFs_selectFields/all.mergedMAF.txt", sep='\t')

In [ ]:
old_wgs_maf = pd.read_csv("gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/20586939-b3c2-4dae-a397-d5ba5a04dddb/WGS_aggregate/6a5580e0-2889-42a6-bbbc-62e1056eb8b2/call-aggregateMAFs_selectFields/allcurrent.mergedMAF.txt", sep='\t')

In [ ]:
old_wes_maf = old_wes_maf.rename(columns={"Tumor_Sample_Barcode": "DepMap_ID"})
old_wgs_maf = old_wgs_maf.rename(columns={"Tumor_Sample_Barcode": "DepMap_ID"})
old_wgs_maf = old_wgs_maf[old_wes_maf.columns]

In [ ]:
merged_old = pd.concat([old_wes_maf, old_wgs_maf])
merged_old.to_csv("old_maf.csv", index=None)

In [ ]:
merged_old

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

tc.update_dataset(
    changes_description="subset old maf columns",
    dataset_permaname="comparison-new-maf-old-maf-24b4",
    upload_files=[            
        {
            "path": "old_maf.csv",
            "format": "TableCSV",
            "encoding": "utf-8",
        },
    ],
    upload_async=False,
    add_all_existing_files=True,
)

In [ ]:
tc.update_dataset(
    changes_description="drop unnamed col",
    dataset_permaname="comparison-new-maf-old-maf-24b4",
    upload_files=[            
        {
            "path": "old_maf.csv",
            "format": "TableCSV",
            "encoding": "utf-8",
        },
        {
            "path": "new_maf.csv",
            "format": "TableCSV",
            "encoding": "utf-8",
        },
    ],
    upload_async=False,
    add_all_existing_files=True,
)

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()
new = tc.get(name='comparison-new-maf-old-maf-24b4', version=3, file='new')
new_maf = tc.get(name='comparison-new-maf-old-maf-24b4', version=3, file='new_maf')

In [ ]:
set(new.columns) - set(new_maf.columns)

In [ ]:
old_maf = tc.get(name='comparison-new-maf-old-maf-24b4', version=9, file='old_maf')
old_maf